# James Jones
### 11-17-2022

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import os,json

### I. Clean the files and combine them into one final DataFrame.

- This dataframe should have the following columns:
  - Hero (Just the name of the Hero)
  - Publisher
  - Gender
  - Eye color
  - Race
  - Hair color
  - Height (numeric)
  - Skin color
  - Alignment
  - Weight (numeric)
  
- Plus, one-hot-encoded columns for every power that appears in the dataset. E.g.: **(will be done at the end)**
  - Agility
  - Flight
  - Superspeed

In [2]:
# Load in super hero info DF
df_i = pd.read_csv('Data/superhero_info - superhero_info.csv')
df_i.head()

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}"


In [3]:
# Load in super hero powers DF
df_p = pd.read_csv('Data/superhero_powers - superhero_powers.csv')
df_p.head()

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


## Begin cleaning, splitting, etc. so we have no duplicates and we have only one type of value per column (ex. split Hero|Publisher)

In [4]:
df_i.duplicated().sum()

0

In [5]:
df_p.duplicated().sum()

0

In [6]:
df_i.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Hero|Publisher  463 non-null    object
 1   Gender          463 non-null    object
 2   Race            463 non-null    object
 3   Alignment       463 non-null    object
 4   Hair color      463 non-null    object
 5   Eye color       463 non-null    object
 6   Skin color      463 non-null    object
 7   Measurements    463 non-null    object
dtypes: object(8)
memory usage: 29.1+ KB


In [7]:
df_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   hero_names  667 non-null    object
 1   Powers      667 non-null    object
dtypes: object(2)
memory usage: 10.5+ KB


In [8]:
print(df_i.isna().sum())
print(df_p.isna().sum())

Hero|Publisher    0
Gender            0
Race              0
Alignment         0
Hair color        0
Eye color         0
Skin color        0
Measurements      0
dtype: int64
hero_names    0
Powers        0
dtype: int64


### These are free of duplicates and null/missing values

In [9]:
# Start splitting columns in df_i
df_i['Hero|Publisher'].head(3)

0            A-Bomb|Marvel Comics
1    Abe Sapien|Dark Horse Comics
2              Abin Sur|DC Comics
Name: Hero|Publisher, dtype: object

In [10]:
# Need to split on the "|" using expand = True
df_i['Hero|Publisher'].str.split('|', expand = True)

,0,1
0,A-Bomb,Marvel Comics
1,Abe Sapien,Dark Horse Comics
2,Abin Sur,DC Comics
3,Abomination,Marvel Comics
4,Absorbing Man,Marvel Comics
...,...,...
458,Yellowjacket,Marvel Comics
459,Yellowjacket II,Marvel Comics
460,Yoda,George Lucas
461,Zatanna,DC Comics


In [11]:
# Save our 2 new columns into the dataframe
df_i[['Hero', 'Publisher']] = df_i['Hero|Publisher'].str.split('|',
                                                                expand = True)
df_i.head(3)

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}",Abin Sur,DC Comics


In [12]:
# Now eliminate original Hero|Publisher column
df_i = df_i.drop(columns = 'Hero|Publisher')
df_i.head(3)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics
2,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}",Abin Sur,DC Comics


In [13]:
# Let's examine the a sample of the Measurments column
measure = df_i.loc[0,"Measurements"]
print(type(measure))
measure

<class 'str'>


"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"

In [14]:
# Need to approach this like a str, and not a dict
    # Test run
measure = measure.replace("'", '"') # Look closely, this is hard to read
measure

'{"Height": "203.0 cm", "Weight": "441.0 kg"}'

In [15]:
# Now we can use json.loads
fixed_measure = json.loads(measure)
print(type(fixed_measure))
fixed_measure

<class 'dict'>


{'Height': '203.0 cm', 'Weight': '441.0 kg'}

In [16]:
# Now that we know this will work, we can apply it to the entire column

df_i['Measurements'] = df_i['Measurements'].str.replace("'", '"')

# Apply the json.loads to the full column
df_i['Measurements'] = df_i['Measurements'].apply(json.loads)

df_i['Measurements'].head()

0    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
1     {'Height': '191.0 cm', 'Weight': '65.0 kg'}
2     {'Height': '185.0 cm', 'Weight': '90.0 kg'}
3    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
4    {'Height': '193.0 cm', 'Weight': '122.0 kg'}
Name: Measurements, dtype: object

In [17]:
# Check a single value after transformation
test_measure = df_i.loc[0, 'Measurements']
print(type(test_measure))
test_measure

<class 'dict'>


{'Height': '203.0 cm', 'Weight': '441.0 kg'}

In [18]:
# Unpack measurments column (dictionaries) into separate columns
measurements = df_i['Measurements'].apply(pd.Series)
measurements

,Height,Weight
0,203.0 cm,441.0 kg
1,191.0 cm,65.0 kg
2,185.0 cm,90.0 kg
3,203.0 cm,441.0 kg
4,193.0 cm,122.0 kg
...,...,...
458,183.0 cm,83.0 kg
459,165.0 cm,52.0 kg
460,66.0 cm,17.0 kg
461,170.0 cm,57.0 kg


In [19]:
# Concatinate this back onto the df_i; inspect
df_i = pd.concat((df_i, measurements), axis = 1)
df_i.head(3)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg
2,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}",Abin Sur,DC Comics,185.0 cm,90.0 kg


In [20]:
# Drop original 'Measurements' column
df_i = df_i.drop(columns = 'Measurements')

In [21]:
df_i.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0 cm,90.0 kg
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0 cm,441.0 kg
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0 cm,122.0 kg


In [22]:
# We need to expand "Height" and "Weight" to get just the numeric values
df_i['Height'].str.split(' ', expand = True)

,0,1
0,203.0,cm
1,191.0,cm
2,185.0,cm
3,203.0,cm
4,193.0,cm
...,...,...
458,183.0,cm
459,165.0,cm
460,66.0,cm
461,170.0,cm


In [23]:
# Save our 2 new columns into the dataframe
df_i[['Height(cm)', 'CM']] = df_i['Height'].str.split(' ', expand = True)
df_i.head(3)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,Height(cm),CM
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0 cm,441.0 kg,203.0,cm
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg,191.0,cm
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0 cm,90.0 kg,185.0,cm


In [24]:
# Repeat with our "Weight"
df_i[['Weight(kg)', 'KG']] = df_i['Weight'].str.split(' ', expand = True)
df_i.head(3)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,Height(cm),CM,Weight(kg),KG
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0 cm,441.0 kg,203.0,cm,441.0,kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg,191.0,cm,65.0,kg
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0 cm,90.0 kg,185.0,cm,90.0,kg


In [25]:
# Drop 'CM', 'KG', 'Height', and 'Weight' columns
df_i = df_i.drop(columns = ['CM', 'KG', 'Height', 'Weight'])
df_i.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height(cm),Weight(kg)
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0


In [26]:
# Change Heigh(cm) and Weight(kg) to floats
df_i['Height(cm)'] = df_i['Height(cm)'].astype(float)
df_i['Weight(kg)'] = df_i['Weight(kg)'].astype(float)
df_i.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Gender      463 non-null    object 
 1   Race        463 non-null    object 
 2   Alignment   463 non-null    object 
 3   Hair color  463 non-null    object 
 4   Eye color   463 non-null    object 
 5   Skin color  463 non-null    object 
 6   Hero        463 non-null    object 
 7   Publisher   463 non-null    object 
 8   Height(cm)  463 non-null    float64
 9   Weight(kg)  463 non-null    float64
dtypes: float64(2), object(8)
memory usage: 36.3+ KB


### We now have a clean info DF (and it matches our assignment parameters) let's take a second look at our powers DF, and prep the powers for OHE

In [27]:
df_p.head()

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


### We need to separate out all of our poweres to be able to One Hot Encode

In [28]:
df_p['powers_split'] = df_p['Powers'].str.split(",")
df_p.head()

,hero_names,Powers,powers_split
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed","[Agility, Super Strength, Stamina, Super Speed]"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...","[Accelerated Healing, Durability, Longevity, S..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du...","[Agility, Accelerated Healing, Cold Resistance..."
3,Abin Sur,Lantern Power Ring,[Lantern Power Ring]
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt...","[Accelerated Healing, Intelligence, Super Stre..."


In [29]:
# Exploding the column of lists
exploded = df_p.explode('powers_split')
exploded[['hero_names', 'Powers', 'powers_split']].head(5)

,hero_names,Powers,powers_split
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Agility
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Super Strength
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Stamina
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Super Speed
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...",Accelerated Healing


In [30]:
# Saving the unique values from the exploded column
cols_to_make = exploded['powers_split'].dropna().unique()
cols_to_make

array(['Agility', 'Super Strength', 'Stamina', 'Super Speed',
       'Accelerated Healing', 'Durability', 'Longevity', 'Camouflage',
       'Self-Sustenance', 'Cold Resistance', 'Underwater breathing',
       'Marksmanship', 'Weapons Master', 'Intelligence', 'Telepathy',
       'Immortality', 'Reflexes', 'Enhanced Sight', 'Sub-Mariner',
       'Lantern Power Ring', 'Invulnerability', 'Animation',
       'Super Breath', 'Dimensional Awareness', 'Flight', 'Size Changing',
       'Teleportation', 'Magic', 'Dimensional Travel',
       'Molecular Manipulation', 'Energy Manipulation', 'Power Cosmic',
       'Energy Absorption', 'Elemental Transmogrification',
       'Fire Resistance', 'Natural Armor', 'Heat Resistance',
       'Matter Absorption', 'Regeneration', 'Stealth', 'Power Suit',
       'Energy Blasts', 'Energy Beams', 'Heat Generation', 'Danger Sense',
       'Phasing', 'Force Fields', 'Hypnokinesis', 'Invisibility',
       'Enhanced Senses', 'Jump', 'Shapeshifting', 'Elasticity',
 

In [31]:
for col in cols_to_make:
    df_p[col] = df_p['Powers'].str.contains(col)

df_p.head(3)

/var/folders/sh/hqffbqw50l9fmqy7fht31tx00000gn/T/ipykernel_5385/982269933.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_p[col] = df_p['Powers'].str.contains(col)
/var/folders/sh/hqffbqw50l9fmqy7fht31tx00000gn/T/ipykernel_5385/982269933.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_p[col] = df_p['Powers'].str.contains(col)
/var/folders/sh/hqffbqw50l9fmqy7fht31tx00000gn/T/ipykernel_5385/982269933.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many t

,hero_names,Powers,powers_split,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed","[Agility, Super Strength, Stamina, Super Speed]",True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...","[Accelerated Healing, Durability, Longevity, S...",False,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du...","[Agility, Accelerated Healing, Cold Resistance...",True,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False


### Now tha we have all of our powers expended, we can connec the 2 DFs together by our Super Hero Names

In [32]:
# Merging with different column names
hero_df = pd.merge(df_i, df_p, left_on = 'Hero', right_on = 'hero_names')

In [33]:
hero_df.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height(cm),Weight(kg),...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,...,False,False,False,False,False,False,False,False,False,False
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0,...,False,False,False,False,False,False,False,False,False,False
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0,...,False,False,False,False,False,False,False,False,False,False
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0,441.0,...,False,False,False,False,False,False,False,False,False,False
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0,122.0,...,False,False,False,False,False,False,False,False,False,False


In [34]:
hero_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 463 entries, 0 to 462
Columns: 180 entries, Gender to Changing Armor
dtypes: bool(167), float64(2), object(11)
memory usage: 126.1+ KB


In [35]:
# Note that powers, hero_names, and powers_split are stin in our dataframe
hero_df = hero_df.drop(columns = ['hero_names', 'Powers', 'powers_split'])

### II. Use your combined DataFrame to answer the following questions.

1. Compare the average weight of super powers who have Super Speed to those who do not.
2. What is the average height of heroes for each publisher?


In [36]:
# Average weight amoung all heros
hero_df['Weight(kg)'].mean()

113.58963282937366

In [37]:
# Locate our Super Speed Heros
super_speed = hero_df.loc[hero_df['Super Speed'] == True]

In [38]:
# Adverage weight of those with super speed
super_speed['Weight(kg)'].mean()

129.40404040404042

In [39]:
# Remove only heros with Super Speed
no_super_speed = hero_df.loc[hero_df['Super Speed'] == False]

In [40]:
no_super_speed['Weight(kg)'].mean()

101.77358490566037


### Average Weight (in kg) of Hero's with Super Speed = 129.4
### Average Weight (in kg) of Hero's without Super Speed = 101.8

In [41]:
# In code
print(f"Average Weight of Hero's with Super Speed: {super_speed['Weight(kg)'].mean()}")
print(f"Average Weight of Hero's without Super Speed: {no_super_speed['Weight(kg)'].mean()}")

Average Weight of Hero's with Super Speed: 129.40404040404042
Average Weight of Hero's without Super Speed: 101.77358490566037


### Now list our Hero's average height by Publisher

In [42]:
hero_df['Publisher'].value_counts()

Marvel Comics        297
DC Comics            138
Dark Horse Comics     11
George Lucas           5
Team Epic TV           4
Shueisha               4
Star Trek              2
Unknown                1
Image Comics           1
Name: Publisher, dtype: int64

In [43]:
hero_df.groupby('Publisher')['Weight(kg)'].mean()

Publisher
DC Comics            104.188406
Dark Horse Comics    101.818182
George Lucas          77.400000
Image Comics         405.000000
Marvel Comics        119.579125
Shueisha              64.500000
Star Trek             79.000000
Team Epic TV          72.000000
Unknown               83.000000
Name: Weight(kg), dtype: float64

### Finally, we can see our Hero's average heigh by publisher